# Loading the Peft Library

In [1]:
!pip install -q peft datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM

# Loading the model and the tokenizers.

In [3]:
model_name = "HuggingFaceTB/SmolLM2-360M"
NUM_VIRTUAL_TOKENS = 35  # These no. of tokens will be added in the pretrained model, which will be trainable

NUM_EPOCHS_PROMPT = 10
NUM_EPOCHS_CLASSIFIER = 10
device = "cuda"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token # Add this line to set the padding token

foundational_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map = device
)

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/724M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

# Inference with the pre trained Smollm2 model

In [5]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=100):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        #temperature=0.2,
        #top_p=0.95,
        #do_sample=True,
        repetition_penalty=1.5, #Avoid repetition.
        early_stopping=True, #The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id
    )
    return outputs

I will run the same sentence on both models in order to compare the pre-trained model with the same model following the prompt-tuning procedure.

In [6]:
import os
from datasets import load_dataset
from peft import  get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit
from transformers import TrainingArguments




In [25]:
input_classifier = tokenizer("Sentence : Because everyone knows this islam men is the devil. Label : ", return_tensors="pt")
foundational_outputs_prompt = get_outputs(foundational_model,
                                          input_classifier.to(device),
                                          max_new_tokens=3)

print(tokenizer.batch_decode(foundational_outputs_prompt, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


['Sentence : Because everyone knows this islam men is the devil. Label : 10\n']


The model is generating ambiguous response and it completes the sentence as best as it can.

In [8]:
dataset_classifier = "SetFit/ethos_binary"

def concatenate_columns_classifier(dataset):
    def concatenate(example):
        example['text'] = "Sentence : {} Label : {}".format(example['text'], example['label_text'])
        return example

    dataset = dataset.map(concatenate)
    return dataset

In [9]:
data_classifier = load_dataset(dataset_classifier)
data_classifier['train'] = concatenate_columns_classifier(
    data_classifier['train'])

data_classifier = data_classifier.map(
    lambda samples: tokenizer(samples["text"]),
    batched=True)
train_sample_classifier = data_classifier["train"].remove_columns(
    ['label', 'label_text', 'text'])

README.md:   0%|          | 0.00/162 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/107k [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/64.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/598 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/598 [00:00<?, ? examples/s]

Map:   0%|          | 0/598 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [10]:
data_classifier

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 598
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 400
    })
})

In [11]:
train_sample_classifier

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 598
})

In [12]:
print(train_sample_classifier[2:3])

{'input_ids': [[35658, 535, 1577, 9413, 314, 260, 3040, 282, 253, 1420, 16587, 24246, 1577, 787, 13735, 4778]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


# Prompt-Tuning Configuration

In [13]:
generation_config_classifier = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,   #This type indicates the model will generate text.
    prompt_tuning_init=PromptTuningInit.TEXT,
    prompt_tuning_init_text="Indicate if the text contains hate speech or no hate speech.",
    #Number of virtual tokens to be added and trained.
    num_virtual_tokens=NUM_VIRTUAL_TOKENS,   # 20 virtual tokens
    #The pre-trained model.
    tokenizer_name_or_path=model_name
)

In [14]:
peft_model_classifier = get_peft_model(
    foundational_model,
    generation_config_classifier)
print(peft_model_classifier.print_trainable_parameters())  # Get the total percentage of trainable parameters

trainable params: 33,600 || all params: 361,854,720 || trainable%: 0.0093
None


**Did you notice the decrease in trainable parameters?** That's incredible. <br>**0.0093% of the available paramaters will be trained.**

In [15]:
import os

working_dir = "/content/Prompt_Tuning_SmolLM2/"

#Is best to store the models in separate folders.
#Create the name of the directories where to store the models.
output_directory_classifier =  os.path.join(working_dir, "peft_outputs_classifier")

#Just creating the directoris if not exist.
if not os.path.exists(working_dir):
    os.mkdir(working_dir)

In [16]:
from transformers import TrainingArguments
def create_training_arguments(path, learning_rate=0.0035, epochs=6, autobatch=True):
    training_args = TrainingArguments(
        output_dir=path, # Where the model predictions and checkpoints will be written
        #use_cpu=True, # This is necessary for CPU clusters.
        auto_find_batch_size=autobatch, # Find a suitable batch size that will fit into memory automatically
        learning_rate= learning_rate, # Higher learning rate than full fine-tuning
        #per_device_train_batch_size=4,
        num_train_epochs=epochs,
        report_to="none"
    )
    return training_args

In [17]:
training_args_classifier = create_training_arguments(
    output_directory_classifier,
    3e-2,
    NUM_EPOCHS_CLASSIFIER)

# Training the Model

In [18]:
from transformers import Trainer, DataCollatorForLanguageModeling
def create_trainer(model, training_args, train_dataset):
    trainer = Trainer(
        model=model, # We pass in the PEFT version of the foundation model, SmolLM2
        args=training_args, #arguments
        train_dataset=train_dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False) # mlm=False indicates not to use masked language modeling
    )
    return trainer

In [19]:
trainer_classifier = create_trainer(peft_model_classifier,
                                   training_args_classifier,
                                   train_sample_classifier)
trainer_classifier.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


Step,Training Loss


Step,Training Loss
500,2.879000
1000,2.851200
1500,2.814500
2000,2.769900
2500,2.745600


TrainOutput(global_step=2990, training_loss=2.7943169533209655, metrics={'train_runtime': 431.2987, 'train_samples_per_second': 13.865, 'train_steps_per_second': 6.933, 'total_flos': 720313241472000.0, 'train_loss': 2.7943169533209655, 'epoch': 10.0})

# Inference Model

In [20]:
trainer_classifier.model.save_pretrained(output_directory_classifier)

In [21]:
from peft import PeftModel

loaded_model_peft = PeftModel.from_pretrained(foundational_model,
                                         output_directory_classifier,
                                         #device_map=device,
                                         is_trainable=False)

In [22]:
loaded_model_peft.load_adapter(output_directory_classifier, adapter_name="classifier")
loaded_model_peft.set_adapter("classifier")

In [24]:
input_classifier = tokenizer("Sentence : Because everyone knows this islam men is the devil.  Label : ", return_tensors="pt").to(device)

loaded_model_sentences_outputs = get_outputs(loaded_model_peft,
                                             input_classifier,
                                             max_new_tokens=3)
print(tokenizer.batch_decode(loaded_model_sentences_outputs, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


['Sentence : Because everyone knows this islam men is the devil.  Label :  hate speech,']
